In [79]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
df = pd.read_csv(r"C:\Users\admin5\Desktop\RFM.csv")
df.head()
df1 = df

In [80]:
customer_country=df1[['CustomerID']].drop_duplicates()
customer_country.groupby(['CustomerID']).aggregate('count').reset_index().sort_values('CustomerID', ascending=False)

,CustomerID
10357,11372
10356,11371
10355,11370
10354,11369
10353,11368
10352,11367
10351,11366
10350,11365
10349,11364
10348,11363


In [81]:
#Check whether there are missing values in each column.

df1.isnull().sum(axis=0)

RetailStore      0
CustomerID       0
TransactionID    0
InvoiceDate      0
ProductID        0
Cost             0
dtype: int64

In [82]:
#remove missing values if any

df1 = df1[pd.notnull(df1['CustomerID'])]


In [83]:
df1.shape


(56798, 6)

In [84]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56798 entries, 0 to 56797
Data columns (total 6 columns):
RetailStore      56798 non-null int64
CustomerID       56798 non-null int64
TransactionID    56798 non-null object
InvoiceDate      56798 non-null object
ProductID        56798 non-null int64
Cost             56798 non-null float64
dtypes: float64(1), int64(3), object(2)
memory usage: 2.6+ MB


# This finish Cleansing the data

In [85]:
#check unique values in each column

def unique_counts(df1):
   for i in df1.columns:
       count = df1[i].nunique()
       print(i, ": ", count)
unique_counts(df1)

RetailStore :  5
CustomerID :  10358
TransactionID :  21729
InvoiceDate :  726
ProductID :  89
Cost :  422


In [86]:
#Find out the first and last order dates in the data.

df1['InvoiceDate'].min()

'1/1/2019'

In [87]:
df1['InvoiceDate'].max()

'9/9/2020'

In [88]:
#Since recency is calculated for a point in time, 
#and the last invoice date is 2020–09–09, 
#we will use 2021–2–2 to calculate recency.

import datetime as dt
NOW = dt.datetime(2021,2,2)
df1['InvoiceDate'] = pd.to_datetime(df1['InvoiceDate'])

# RFM SEGMENTATION

In [90]:
#Creating a RFM table

rfmTable = df1.groupby('CustomerID').agg({'InvoiceDate': lambda x: (NOW - x.max()).days, 'TransactionID': lambda x: len(x), 'Cost': lambda x: x.sum()})
rfmTable['InvoiceDate'] = rfmTable['InvoiceDate'].astype(int)
rfmTable.rename(columns={'InvoiceDate': 'recency', 
                         'TransactionID': 'frequency', 
                         'Cost': 'monetary_value'}, inplace=True)

In [91]:
#Calculate RFM metrics for each customer

rfmTable.head()

,recency,frequency,monetary_value
CustomerID,,,
1001,763,1,144.75
1002,309,16,2162.75
1003,763,2,353.25
1004,321,13,2673.50
1005,763,1,147.25


In [42]:
#Split the metrics
#The easiest way to split metrics into segments is by using quartiles.
#This gives us a starting point for the detailed analysis.
#4 segments are easy to understand and explain.

quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])
quantiles = quantiles.to_dict()

In [92]:
#Create a segmented RFM table

segmented_rfm = rfmTable

In [93]:
#Best customers
#The lowest recency
#The highest frequency
#The highest Monetary value 



def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [94]:
#Add segment numbers to the newly created segmented RFM table

segmented_rfm['r_quartile'] = segmented_rfm['recency'].apply(RScore, args=('recency',quantiles,))
segmented_rfm['f_quartile'] = segmented_rfm['frequency'].apply(FMScore, args=('frequency',quantiles,))
segmented_rfm['m_quartile'] = segmented_rfm['monetary_value'].apply(FMScore, args=('monetary_value',quantiles,))
segmented_rfm.head()

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile
CustomerID,,,,,,
1001,763,1,144.75,4,4,4
1002,309,16,2162.75,4,1,1
1003,763,2,353.25,4,4,3
1004,321,13,2673.50,4,1,1
1005,763,1,147.25,4,4,4


In [95]:
#Adding a new column to combine RFM score: 
#example : 111 is the highest score

segmented_rfm['RFMScore'] = segmented_rfm.r_quartile.map(str) + segmented_rfm.f_quartile.map(str) + segmented_rfm.m_quartile.map(str)
segmented_rfm.head()

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
1001,763,1,144.75,4,4,4,444
1002,309,16,2162.75,4,1,1,411
1003,763,2,353.25,4,4,3,443
1004,321,13,2673.50,4,1,1,411
1005,763,1,147.25,4,4,4,444


In [106]:
#Displaying top 10 cheap customers!

segmented_rfm[segmented_rfm['RFMScore']=='444'].sort_values('monetary_value', ascending=False).head(10)

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
6841,278,2,349.5,4,4,4,444
6434,298,2,349.5,4,4,4,444
5006,430,2,349.5,4,4,4,444
9763,75,2,349.5,4,4,4,444
8387,180,2,349.5,4,4,4,444
11171,330,2,349.5,4,4,4,444
9698,80,2,349.5,4,4,4,444
7677,230,2,349.5,4,4,4,444
8514,169,2,349.5,4,4,4,444
